In [3]:
import pandas as pd
import numpy as np
import re

def extract_best_metrics(df, location, direction):
    satellite_ids = sorted(list(set([col.split('_')[0] for col in df.columns if col.startswith('LEO')])))
    results = []

    for index, row in df.iterrows():
        best = {
            'SAT_ID': None, 'BEST_SNR': -np.inf, 'BEST_RSSI': -np.inf, 'BEST_Thrpt': -np.inf,
            'BEST_BER_MQAM': np.inf, 'BEST_BER_QPSK': np.inf, 'BEST_Latency': np.inf
        }

        for sat_id in satellite_ids:
            prefix = f'{sat_id}_{location}_'

            snr = row.get(f'{prefix}SNR_dB')
            if pd.isna(snr):
                continue

            if snr > best['BEST_SNR']:
                best['SAT_ID'] = sat_id
                best['BEST_SNR'] = snr
                best['BEST_RSSI'] = row.get(f'{prefix}RSSI_dBm')
                best['BEST_Thrpt'] = row.get(f'{prefix}Throughput')
                best['BEST_BER_MQAM'] = row.get(f'{prefix}BER_MQAM')
                best['BEST_BER_QPSK'] = row.get(f'{prefix}BER_QPSK')
                best['BEST_Latency'] = row.get(f'{prefix}Latency')

        results.append({
            'Time': row['Time'],
            f'{location}_{direction}_Best_SAT_ID': best['SAT_ID'],
            f'{location}_{direction}_BEST_SNR': best['BEST_SNR'],
            f'{location}_{direction}_BEST_RSSI': best['BEST_RSSI'],
            f'{location}_{direction}_BEST_Thrpt': best['BEST_Thrpt'] / (1024 * 1024) if pd.notna(best['BEST_Thrpt']) else None,  # Mbps
            f'{location}_{direction}_BEST_BER_MQAM': best['BEST_BER_MQAM'],
            f'{location}_{direction}_BEST_BER_QPSK': best['BEST_BER_QPSK'],
            f'{location}_{direction}_BEST_Latency': best['BEST_Latency'] * 1000 if pd.notna(best['BEST_Latency']) else None  # ms
        })

    return pd.DataFrame(results)

# --- Load original files ---
df_downlink = pd.read_csv('./data/Satellite_Australia_Simulation_Log_starlink_downlink.csv')
df_uplink = pd.read_csv('./data/Satellite_Australia_Simulation_Log_starlink_uplink.csv')

# --- Extract per-location best metrics ---
melbourne_uplink_best = extract_best_metrics(df_uplink, 'Melbourne', 'Uplink')
sydney_downlink_best = extract_best_metrics(df_downlink, 'Sydney', 'Downlink')

# --- Merge on Time ---
combined_best_df = pd.merge(sydney_downlink_best, melbourne_uplink_best, on='Time', how='inner')

# --- Save ---
combined_best_df.to_csv("Combined_UplinkDownlink_Best_Metrics.csv", index=False)
print(combined_best_df.head())


                   Time Sydney_Downlink_Best_SAT_ID  Sydney_Downlink_BEST_SNR  \
0  10-Apr-2025 12:00:00                       LEO35                 -1.502024   
1  10-Apr-2025 12:00:30                       LEO35                  0.014226   
2  10-Apr-2025 12:01:00                       LEO35                  3.138661   
3  10-Apr-2025 12:01:30                       LEO35                  3.955374   
4  10-Apr-2025 12:02:00                       LEO35                  2.197404   

   Sydney_Downlink_BEST_RSSI  Sydney_Downlink_BEST_Thrpt  \
0                -121.453115                  184.055200   
1                -119.936865                  238.982391   
2                -116.812430                  384.695336   
3                -115.995717                  429.548857   
4                -117.753686                  336.328556   

   Sydney_Downlink_BEST_BER_MQAM  Sydney_Downlink_BEST_BER_QPSK  \
0                       0.265039                       0.117095   
1                 

In [4]:
# Compute final metrics
final_df = pd.DataFrame()
final_df['Time'] = combined_best_df['Time']

# Min throughput (Mbps)
final_df['EndToEnd_Thrpt_Mbps'] = combined_best_df[['Melbourne_Uplink_BEST_Thrpt', 'Sydney_Downlink_BEST_Thrpt']].min(axis=1)

# Max BER_QPSK
final_df['EndToEnd_BER_QPSK'] = combined_best_df[['Melbourne_Uplink_BEST_BER_QPSK', 'Sydney_Downlink_BEST_BER_QPSK']].max(axis=1)

# Sum latency (ms)
final_df['EndToEnd_Latency_ms'] = combined_best_df['Melbourne_Uplink_BEST_Latency'] + combined_best_df['Sydney_Downlink_BEST_Latency']

# Optional: include SAT IDs if needed for analysis
final_df['Melbourne_Uplink_Best_SAT_ID'] = combined_best_df['Melbourne_Uplink_Best_SAT_ID']
final_df['Sydney_Downlink_Best_SAT_ID'] = combined_best_df['Sydney_Downlink_Best_SAT_ID']

# Save to CSV
final_df.to_csv("EndToEnd_Starlink_Performance.csv", index=False)

print(final_df.head())


                   Time  EndToEnd_Thrpt_Mbps  EndToEnd_BER_QPSK  \
0  10-Apr-2025 12:00:00           184.055200           0.117095   
1  10-Apr-2025 12:00:30           238.982391           0.078310   
2  10-Apr-2025 12:01:00           235.365395           0.021190   
3  10-Apr-2025 12:01:30           429.548857           0.012877   
4  10-Apr-2025 12:02:00           336.328556           0.034279   

   EndToEnd_Latency_ms Melbourne_Uplink_Best_SAT_ID  \
0             6.116335                        LEO35   
1             6.054165                        LEO35   
2             6.256142                        LEO35   
3             6.308717                        LEO68   
4             5.999658                        LEO68   

  Sydney_Downlink_Best_SAT_ID  
0                       LEO35  
1                       LEO35  
2                       LEO35  
3                       LEO35  
4                       LEO35  
